# Preparation of the drinking water quality data

# Description

```

```

In [1]:
# Imports
from PIL import Image
import pandas as pd
import numpy as np
import os,sys

In [2]:
# Read raw files
raw_path = "./raw_data/terviseamet"
raw_files = os.listdir(raw_path)

original_columns = pd.read_csv(os.path.join(raw_path, raw_files[0]))

raw_filelist = []
for f in raw_files:
    df = pd.read_csv(os.path.join(raw_path, f))
    
    # select columns and translate into english
    col_map = {
        'id': 'test_id',
        'veevark_id': 'station_id', 
        'veeliik': 'water_type', 
        'proovivotu_aeg': 'date',
        'proovivotu_metoodika': 'test_method', 
        'id3': 'indicator_id',
        'nimetus4': 'indicator_name',
        'sisaldus': 'value',
        'yhik': 'unit',
        'hinnang5': 'assessment'
        }
    df = df[col_map.keys()]
    df.rename(columns=col_map, inplace=True)
    
    # Add year variable
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df.date.dt.year

    print(len(df.station_id.unique()))

    # Add df to list for concatenation later
    raw_filelist.append(df)

long_df = pd.concat(raw_filelist, ignore_index=True)
long_df

411
1104


/tmp/ipykernel_332/262506298.py:9: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(raw_path, f))


1189
1063


/tmp/ipykernel_332/262506298.py:9: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(raw_path, f))


1138


/tmp/ipykernel_332/262506298.py:9: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(raw_path, f))


982
1110


/tmp/ipykernel_332/262506298.py:9: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(raw_path, f))


1080
1160


,test_id,station_id,water_type,date,test_method,indicator_id,indicator_name,value,unit,assessment,year
0,276444,302,Joogivesi,2022-05-07,NaN,241,Maitse (lahjendusaste),1,lahjendusaste,vastab nõuetele,2022
1,276444,302,Joogivesi,2022-05-07,NaN,112,Lõhn (lahjendusaste),1,lahjendusaste,vastab nõuetele,2022
2,276444,302,Joogivesi,2022-05-07,NaN,131,Värvus (Pt/Co skaala),3,mg/l Pt,vastab nõuetele,2022
3,276444,302,Joogivesi,2022-05-07,NaN,16,pH,7.7,pH ühik,vastab nõuetele,2022
4,276444,302,Joogivesi,2022-05-07,NaN,51,Elektrijuhtivus,630,μS/cm,vastab nõuetele,2022
...,...,...,...,...,...,...,...,...,...,...,...
504185,173704,306,Joogivesi,2018-01-01,NaN,131,Värvus (Pt/Co skaala),3,mg/l Pt,vastab nõuetele,2018
504186,173704,306,Joogivesi,2018-01-01,NaN,16,pH,7.22,pH ühik,vastab nõuetele,2018
504187,173878,306,Joogivesi,2018-01-01,NaN,7,Clostridium perfringens (koos eostega),0,PMÜ/100ml,vastab nõuetele,2018
504188,173878,306,Joogivesi,2018-01-01,NaN,4,Escherichia coli,0,PMÜ/100 ml,vastab nõuetele,2018


# The last n tests of each station

In [50]:
# Keep only 2017-2021
period_df = long_df[(long_df.year>=2019) & (long_df.year < 2022)]

In [61]:
last_n_tests = long_df.groupby(by=['station_id', 'test_id', 'date']).size()
last_n_tests = last_n_tests.to_frame().reset_index()
last_n_tests = last_n_tests.sort_values(by='date').groupby(by='station_id').apply(lambda x: x.iloc[-4:])
last_n_tests = last_n_tests.reset_index(drop=True)[['station_id', 'test_id','date']]
last_n_tests
#last_n_tests.to_frame().sort_index()[-3:]
# dfs = last_n_tests.groups.items()
#last_n_tests.apply(lambda x: x.iloc[-3:])

,station_id,test_id,date
0,162,206725,2019-11-28 00:00:00
1,162,206809,2019-11-28 00:00:00
2,162,259796,2021-01-12 00:00:00
3,162,259745,2021-01-12 00:00:00
4,163,242150,2021-12-05 10:50:00
...,...,...,...
5240,2676,274169,2022-01-06 09:58:00
5241,2678,256138,2021-10-18 00:00:00
5242,2678,256134,2021-10-18 00:00:00
5243,2678,256136,2021-10-18 00:00:00


In [3]:
long_df.groupby(by=['year','assessment']).size()

year  assessment                    
2014  Halvem kui kvaliteediklass III        1
      Kvaliteediklass I                    41
      ei vasta nõuetele                   690
      vastab nõuetele                   69970
2015  Kvaliteediklass I                    10
      ei vasta nõuetele                   573
      vastab nõuetele                   71163
2016  Kvaliteediklass I                     4
      ei vasta nõuetele                   593
      vastab nõuetele                   72709
2017  ei vasta nõuetele                   607
      vastab nõuetele                   71393
2018  ei vasta nõuetele                   443
      vastab nõuetele                   52280
2019  ei vasta nõuetele                   352
      vastab nõuetele                   33635
2020  ei vasta nõuetele                   386
      vastab nõuetele                   56834
2021  ei vasta nõuetele                   468
      vastab nõuetele                   58411
2022  ei vasta nõuetele                    

In [4]:
# How many tests per year per station_id
long_df.groupby(by=['station_id', 'year']).test_id.unique()

station_id  year
162         2014                                [80238, 80290, 61616]
            2015                              [106003, 106041, 98823]
            2016                             [133003, 131795, 122653]
            2017                                     [137400, 137538]
            2018    [182057, 182240, 177174, 177336, 168987, 16925...
                                          ...                        
2604        2022                                     [268222, 268532]
2643        2022                                             [274479]
2676        2022                                     [274169, 274168]
2678        2021                             [256134, 256136, 256138]
2679        2022                                             [275717]
Name: test_id, Length: 9237, dtype: object

In [10]:
long_df.value.dtype

dtype('O')

# What indicators to keep

In [6]:
# Keep only 2017-2021
period_df = long_df[(long_df.year>=2019) & (long_df.year < 2022)]

In [27]:
# Which indicators exist for all years
inds_per_year_df = period_df.groupby(by=['indicator_name', 'year']).size().sort_values(ascending=False).to_frame(name='size')
inds_per_year_wide = inds_per_year_df.pivot_table(
    index = ['indicator_name'],
    columns = 'year',
    values = 'size'
)

# Indicators that have more than n instances for all years
inds_that_exist_all_years = inds_per_year_wide.dropna()
for y in period_df.year.unique():
    inds_that_exist_all_years = inds_that_exist_all_years[inds_that_exist_all_years[y]>=200]
indicators_to_keep = inds_that_exist_all_years.index
indicators_to_keep

Index(['Alumiinium', 'Ammoonium', 'Boor', 'Coli-laadsed bakterid',
       'Coli-laadsed bakterid (Colilert)', 'Elektrijuhtivus', 'Enterokokid',
       'Escherichia coli', 'Escherichia coli (Colilert)', 'Fluoriid',
       'Hägusus (NTU)', 'Kloriid', 'Kolooniate arv 22 °C',
       'Lõhn (lahjendusaste)', 'Lõhn (pallides)', 'Maitse (lahjendusaste)',
       'Maitse (pallides)', 'Mangaan', 'Naatrium', 'Nitraat', 'Nitrit',
       'Oksüdeeritavus', 'Raud', 'Sulfaat', 'Värvus (Pt-Co ühik)',
       'Värvus (Pt/Co skaala)', 'pH'],
      dtype='object', name='indicator_name')

In [28]:
# Dataframe with only top indicators
rows_to_keep = [iname in indicators_to_keep for iname in period_df.indicator_name]
top_ind_df = period_df[rows_to_keep] 
top_ind_df

,test_id,station_id,water_type,date,test_method,indicator_id,indicator_name,value,unit,assessment,year
13627,212401,1724,Joogivesi,2019-12-31 00:00:00,NaN,40,Kolooniate arv 22 °C,87,PMÜ/1 ml,vastab nõuetele,2019
13628,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",112,Lõhn (lahjendusaste),1,lahjendusaste,vastab nõuetele,2019
13629,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",56,Hägusus (NTU),1.0,NTU,vastab nõuetele,2019
13630,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",1,Raud,18,µg/l,vastab nõuetele,2019
13631,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",114,Oksüdeeritavus,4.0,mg/l O2,vastab nõuetele,2019
...,...,...,...,...,...,...,...,...,...,...,...
379714,247588,306,Joogivesi,2021-03-01 00:00:00,NaN,736,Coli-laadsed bakterid (Colilert),0,MPN/100 ml,vastab nõuetele,2021
379716,247587,306,Joogivesi,2021-02-01 00:00:00,NaN,736,Coli-laadsed bakterid (Colilert),0,MPN/100 ml,vastab nõuetele,2021
379717,247587,306,Joogivesi,2021-02-01 00:00:00,NaN,737,Escherichia coli (Colilert),0,MPN/100 ml,vastab nõuetele,2021
379718,247586,306,Joogivesi,2021-01-01 00:00:00,NaN,737,Escherichia coli (Colilert),0,MPN/100 ml,vastab nõuetele,2021


## Tests and stations that 

In [29]:
# Which tests in 2021 didn't pass the evaluation
test_compliance_df = top_ind_df[top_ind_df.year == 2021][['test_id', 'station_id', 'assessment']].assign(count=1).pivot_table(
    values = 'count',
    index = ['test_id', 'station_id'],
    columns = 'assessment',
    aggfunc = 'sum'
)

test_compliance_tmp = ['non-compliant' if val >= 1 else 'compliant' for val in test_compliance_df['ei vasta nõuetele']]
test_compliance_df['compliance'] = test_compliance_tmp
test_compliance_df.reset_index(inplace=True)

# If any test for station is non-compliant then non compliant
compliance_df = test_compliance_df.assign(count=1).pivot_table(
    values = 'count',
    index = ['station_id'],
    columns = 'compliance',
    aggfunc = 'sum'
)
compliance_df
compliance_tmp = [1 if val >= 1 else 0 for val in compliance_df['non-compliant']]
compliance_df['compliance'] = compliance_tmp
compliance_df.reset_index(inplace=True)
compliance_df.groupby(by='compliance').size()


compliance
0    922
1    187
dtype: int64

In [30]:
# Pivot table for test & indicator
single_test_wide = top_ind_df[['test_id', 'indicator_name', 'value']].pivot_table(
    index = 'test_id',
    columns = 'indicator_name',
    values = 'value'
)

/tmp/ipykernel_332/1162552586.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  single_test_wide = top_ind_df[['test_id', 'indicator_name', 'value']].pivot_table(


In [31]:
# How many measurements per station per year
ind_by_year = top_ind_df.groupby(by=['indicator_name', 'year']).size()

In [32]:
# Value column to numeric and errors to NaN
top_ind_df = top_ind_df.assign(value = pd.to_numeric(top_ind_df['value'], errors='coerce'))
top_ind_df

,test_id,station_id,water_type,date,test_method,indicator_id,indicator_name,value,unit,assessment,year
13627,212401,1724,Joogivesi,2019-12-31 00:00:00,NaN,40,Kolooniate arv 22 °C,87.0,PMÜ/1 ml,vastab nõuetele,2019
13628,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",112,Lõhn (lahjendusaste),1.0,lahjendusaste,vastab nõuetele,2019
13629,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",56,Hägusus (NTU),1.0,NTU,vastab nõuetele,2019
13630,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",1,Raud,18.0,µg/l,vastab nõuetele,2019
13631,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",114,Oksüdeeritavus,4.0,mg/l O2,vastab nõuetele,2019
...,...,...,...,...,...,...,...,...,...,...,...
379714,247588,306,Joogivesi,2021-03-01 00:00:00,NaN,736,Coli-laadsed bakterid (Colilert),0.0,MPN/100 ml,vastab nõuetele,2021
379716,247587,306,Joogivesi,2021-02-01 00:00:00,NaN,736,Coli-laadsed bakterid (Colilert),0.0,MPN/100 ml,vastab nõuetele,2021
379717,247587,306,Joogivesi,2021-02-01 00:00:00,NaN,737,Escherichia coli (Colilert),0.0,MPN/100 ml,vastab nõuetele,2021
379718,247586,306,Joogivesi,2021-01-01 00:00:00,NaN,737,Escherichia coli (Colilert),0.0,MPN/100 ml,vastab nõuetele,2021


# Wide data

In [35]:
est = ['1,2-dikloroetaan', 'Alumiinium', 'Ammoonium', 'Antimon', 'Arseen',
       'Benseen', 'Benso(a)püreen', 'Boor', 'Coli-laadsed bakterid',
       'Coli-laadsed bakterid (Colilert)', 'Dieldriin', 'Elavhõbe',
       'Elektrijuhtivus', 'Endosulfaansulfaat', 'Endriin', 'Enterokokid',
       'Escherichia coli', 'Escherichia coli (Colilert)', 'Fluoriid',
       'HCH, alfa', 'HCH, beeta', 'HCH, gamma (Lindaan)',
       'Heksakloorbenseen (HCB)', 'Heptakloor', 'Hägusus (NTU)', 'Kaadmium',
       'Kloriid', 'Kolooniate arv 22 °C', 'Kroom', 'Lõhn (lahjendusaste)',
       'Lõhn (pallides)', 'Maitse (lahjendusaste)', 'Maitse (pallides)',
       'Mangaan', 'Naatrium', 'Nikkel', 'Nitraat', 'Nitrit', 'Oksüdeeritavus',
       'PAH-d summa', 'Pestitsiidide summa', 'Plii', 'Raud', 'Seleen',
       'Sulfaat', 'Tetrakloroeteen ja trikloroeteen', 'Trihalometaanide summa',
       'Tsüaniid', 'Vask', 'Värvus (Pt-Co ühik)', 'Värvus (Pt/Co skaala)',
       'alfa-Endosulfaan', 'beeta-Endosulfaan', 'cis-Heptakloorepoksiid',
       'o,p-DDD', 'o,p-DDE', 'o,p-DDT', 'p,p-DDD', 'p,p-DDE', 'p,p-DDT', 'pH',
       'trans-Heptakloorepoksiid', 'Üldkaredus (mg-ekv)']
eng = ['1-2-dichloroethane', 'Aluminium', 'Ammonium', 'Antimony', 'Arsenic',
       'Benzene', 'Benzoapyrene', 'Boron', 'Coli-like-bacteria',
       'Coli-like-bacteria-Colilert', 'Dieldrin', 'Mercury',
       'Electrical-conductivity', 'Endosulfan-sulfate', 'Endrin', 'Enterococci',
       'Escherichia-coli', 'Escherichia-coli-Colilert', 'Fluoride',
       'HCH, alpha', 'HCH, beta', 'HCH, gamma-Lindan',
       'Hexachlorobenzene-HCB', 'Heptachlor', 'Turbidity-NTU', 'Cadmium',
       'Chloride', 'Colony-count-at-22-C', 'Chromium', 'Odour-dilution-level',
       'Smell-ball-units', 'Taste-dilution-degree', 'Taste-ball-units',
       'Manganese', 'Sodium', 'Nickel', 'Nitrate', 'Nitrite', 'Oxidability',
       'Amount-of-PAHs', 'Amount-of-pesticides', 'Lead', 'Iron', 'Selenium',
       'Sulphate', 'Tetrachloroethene-and-trichloroethene', 'Sum-of-trihalomethanes',
       'Cyanide', 'Copper', 'Color-Pt-Co-unit', 'Color-Pt/Co-scale',
       'alpha-Endosulfan', 'beta-Endosulfan', 'cis-Heptachlorepoxide',
       'o_p-DDD', 'o_p-DDE', 'o_p-DDT', 'p_p-DDD', 'p_p-DDE', 'p_p-DDT', 'pH ',
       'trans-Heptachlor-epoxide', 'Total-hardness-mg-eq']
indicator_name_map = {e:en for e,en in zip(est,eng)}
indicator_name_map

{'1,2-dikloroetaan': '1-2-dichloroethane',
 'Alumiinium': 'Aluminium',
 'Ammoonium': 'Ammonium',
 'Antimon': 'Antimony',
 'Arseen': 'Arsenic',
 'Benseen': 'Benzene',
 'Benso(a)püreen': 'Benzoapyrene',
 'Boor': 'Boron',
 'Coli-laadsed bakterid': 'Coli-like-bacteria',
 'Coli-laadsed bakterid (Colilert)': 'Coli-like-bacteria-Colilert',
 'Dieldriin': 'Dieldrin',
 'Elavhõbe': 'Mercury',
 'Elektrijuhtivus': 'Electrical-conductivity',
 'Endosulfaansulfaat': 'Endosulfan-sulfate',
 'Endriin': 'Endrin',
 'Enterokokid': 'Enterococci',
 'Escherichia coli': 'Escherichia-coli',
 'Escherichia coli (Colilert)': 'Escherichia-coli-Colilert',
 'Fluoriid': 'Fluoride',
 'HCH, alfa': 'HCH, alpha',
 'HCH, beeta': 'HCH, beta',
 'HCH, gamma (Lindaan)': 'HCH, gamma-Lindan',
 'Heksakloorbenseen (HCB)': 'Hexachlorobenzene-HCB',
 'Heptakloor': 'Heptachlor',
 'Hägusus (NTU)': 'Turbidity-NTU',
 'Kaadmium': 'Cadmium',
 'Kloriid': 'Chloride',
 'Kolooniate arv 22 °C': 'Colony-count-at-22-C',
 'Kroom': 'Chromium',
 'Lõh

In [37]:
# Make indicator names more readable
new_names = [indicator_name_map[n] for n in top_ind_df.indicator_name]

to_keep_df = top_ind_df.copy(deep=True)

to_keep_df = top_ind_df.assign(indicator_name = new_names)


# Keep only 2019 and 2020 as variables
to_keep_df = to_keep_df[to_keep_df.year < 2021]

# Create new variable representing indicator and year
to_keep_df['year'] = to_keep_df.year.apply(str)
to_keep_df = to_keep_df.assign(indyear = lambda x:  x.indicator_name + "_" + x.year)

# Make variable with both year and name in it
to_keep_df

,test_id,station_id,water_type,date,test_method,indicator_id,indicator_name,value,unit,assessment,year,indyear
13627,212401,1724,Joogivesi,2019-12-31 00:00:00,NaN,40,Colony-count-at-22-C,87.00,PMÜ/1 ml,vastab nõuetele,2019,Colony-count-at-22-C_2019
13628,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",112,Odour-dilution-level,1.00,lahjendusaste,vastab nõuetele,2019,Odour-dilution-level_2019
13629,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",56,Turbidity-NTU,1.00,NTU,vastab nõuetele,2019,Turbidity-NTU_2019
13630,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",1,Iron,18.00,µg/l,vastab nõuetele,2019,Iron_2019
13631,208314,306,Joogivesi,2019-12-30 08:30:00,"EVS-ISO 5667-5, EVS-EN ISO 19458",114,Oxidability,4.00,mg/l O2,vastab nõuetele,2019,Oxidability_2019
...,...,...,...,...,...,...,...,...,...,...,...,...
176828,219341,306,Joogivesi,2020-02-01 00:00:00,NaN,736,Coli-like-bacteria-Colilert,0.00,MPN/100 ml,vastab nõuetele,2020,Coli-like-bacteria-Colilert_2020
176829,219341,306,Joogivesi,2020-02-01 00:00:00,NaN,56,Turbidity-NTU,0.38,NTU,vastab nõuetele,2020,Turbidity-NTU_2020
176830,219341,306,Joogivesi,2020-02-01 00:00:00,NaN,131,Color-Pt/Co-scale,3.00,mg/l Pt,vastab nõuetele,2020,Color-Pt/Co-scale_2020
176831,219106,306,Joogivesi,2020-01-01 00:00:00,NaN,736,Coli-like-bacteria-Colilert,0.00,MPN/100 ml,vastab nõuetele,2020,Coli-like-bacteria-Colilert_2020


In [143]:
# Keep only relevant variables
wide_df = to_keep_df.pivot_table(
    index = ['station_id'],
    columns = 'indyear',
    values = 'value',
    aggfunc = np.nanmax
)

In [159]:
# Keep rows with max N missing values
n = int(wide_df.shape[1]/2)
missing_count = wide_df.isna().T.sum()
max_n_missing = missing_count[missing_count <= n].to_frame()

In [160]:
# Keep only rows with not more than max allowed number of missing values
clean_df = wide_df[[i in max_n_missing.index for i in wide_df.index]]
clean_df.reset_index(inplace=True)
clean_df

indyear,station_id,Ammoonium_2019,Ammoonium_2020,Coli-laadsed bakterid_2019,Coli-laadsed bakterid_2020,Elektrijuhtivus_2019,Elektrijuhtivus_2020,Escherichia coli_2019,Escherichia coli_2020,Hägusus (NTU)_2019,...,Lõhn (lahjendusaste)_2019,Lõhn (lahjendusaste)_2020,Maitse (lahjendusaste)_2019,Maitse (lahjendusaste)_2020,Raud_2019,Raud_2020,Värvus (Pt/Co skaala)_2019,Värvus (Pt/Co skaala)_2020,pH_2019,pH_2020
0,163,0.08,0.08,0.0,0.0,716.0,671.0,0.0,0.0,1.18,...,1.0,1.0,1.0,1.0,109.4,98.0,11.7,12.1,8.17,8.12
1,165,0.08,0.08,0.0,0.0,1167.0,1170.0,0.0,0.0,1.52,...,1.0,1.0,1.0,1.0,52.9,74.0,7.0,7.8,7.84,7.91
2,167,0.08,0.08,0.0,0.0,996.0,1010.0,0.0,0.0,3.90,...,1.0,3.0,1.0,3.0,540.0,78.5,7.7,9.1,7.83,7.81
3,169,0.29,0.23,0.0,0.0,536.0,550.0,0.0,0.0,0.30,...,NaN,NaN,NaN,NaN,55.0,45.0,NaN,NaN,8.10,8.20
4,170,0.25,0.24,0.0,0.0,631.0,662.0,0.0,0.0,0.62,...,5.0,2.0,5.0,2.0,NaN,NaN,8.8,7.0,7.74,7.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,2305,NaN,NaN,0.0,NaN,805.0,846.0,0.0,NaN,6.70,...,NaN,NaN,NaN,NaN,2100.0,10.0,7.0,3.8,7.60,7.40
899,2307,0.53,NaN,NaN,NaN,742.0,542.0,NaN,NaN,1.00,...,2.0,4.0,2.0,4.0,26.0,410.0,2.0,9.0,7.20,8.00
900,2311,0.22,NaN,NaN,NaN,457.0,725.0,NaN,NaN,1.60,...,2.0,2.0,2.0,2.0,150.0,20.0,2.0,3.0,7.60,7.40
901,2314,NaN,0.05,NaN,0.0,NaN,332.0,NaN,0.0,NaN,...,NaN,1.0,NaN,1.0,NaN,20.0,NaN,0.0,NaN,7.40


In [161]:
# Add results
withresults_df = clean_df.merge(compliance_df[['station_id','compliance']], on='station_id')

In [162]:
# The average value of variables based on compliance 
withresults_df.groupby('compliance').mean()

,station_id,Ammoonium_2019,Ammoonium_2020,Coli-laadsed bakterid_2019,Coli-laadsed bakterid_2020,Elektrijuhtivus_2019,Elektrijuhtivus_2020,Escherichia coli_2019,Escherichia coli_2020,Hägusus (NTU)_2019,...,Lõhn (lahjendusaste)_2019,Lõhn (lahjendusaste)_2020,Maitse (lahjendusaste)_2019,Maitse (lahjendusaste)_2020,Raud_2019,Raud_2020,Värvus (Pt/Co skaala)_2019,Värvus (Pt/Co skaala)_2020,pH_2019,pH_2020
compliance,,,,,,,,,,,,,,,,,,,,,
0,878.597113,0.116303,0.111263,0.851180,0.623377,569.367838,575.379839,0.061706,0.03352,1.216421,...,1.553345,1.666078,1.514620,1.597353,122.299297,121.988527,4.056171,4.227597,7.568558,7.622282
1,977.271605,0.289569,0.245055,3.196429,3.061224,594.857143,604.929114,0.000000,0.00000,1.947143,...,1.625000,1.753846,1.561404,1.807018,238.109455,224.331034,6.067123,6.090541,7.580921,7.669114


# Train and test data

In [173]:
# Rename compliance column
withresults_df.rename(columns={'compliance':'result2021'}, inplace=True)

# Select 70% of data for training
train_df = withresults_df.sample(n = int(withresults_df.shape[0] * 0.7))

# Test data & solution
test_df = withresults_df.loc[~withresults_df.index.isin(train_df.index)]

solution_df = test_df.copy(deep=True)
test_df.pop('result2021')

# Sample submission
result_vals = train_df['result2021'].unique()
sample_submission_df = solution_df.copy(deep=True)[['station_id', 'result2021']]
sample_submission_df['result2021'] = np.random.choice(result_vals, solution_df.shape[0])

In [164]:
# Write data
result_path = "./prepped_data/terviseamet"

train_df.to_csv(os.path.join(result_path, 'train.csv'), index=False)
test_df.to_csv(os.path.join(result_path, 'test.csv'), index=False)
solution_df.to_csv(os.path.join(result_path, 'solution.csv'), index=False)
sample_submission_df.to_csv(os.path.join(result_path, 'sample_submission.csv'), index=False)

In [165]:
# The average value of variables based on compliance 
train_df.groupby('result2021').size()

result2021
0    533
1     57
dtype: int64

In [166]:
# The average value of variables based on compliance 
solution_df.groupby('result2021').size()

result2021
0    229
1     24
dtype: int64

In [167]:
# Check that sample objects doesn't match other objects
sample_submission_df['result2021'] == solution_df['result2021']

0       True
4      False
11     False
12     False
13     False
       ...  
825    False
831    False
835     True
838    False
839     True
Name: result2021, Length: 253, dtype: bool

In [168]:
# Balanced classes training set
train_balanced = train_df.sort_values(by='result2021')
train_balanced = pd.concat([train_balanced.iloc[:70], train_balanced.iloc[-70:]])

train_balanced.groupby('result2021').size()

result2021
0    83
1    57
dtype: int64

In [169]:
# Try to train the model
import sklearn as sk

train_dat = train_balanced
test_dat = solution_df
train_dat.fillna(0, inplace=True)
train_result = train_dat.pop('result2021')
test_dat.fillna(0, inplace=True)
test_result = test_dat.pop('result2021')


In [170]:
from sklearn.tree import DecisionTreeClassifier

# Treenime mudeli, kasutades treenimise ettenähtud andmeid
model = DecisionTreeClassifier(max_depth=5)
model.fit(X=train_dat, y=train_result)

# Teeme prognoosi testandmete peal
tree_prediction = model.predict(X=test_dat)



In [171]:
pred_results = solution_df.assign(
    actual = test_result, 
    predicted = tree_prediction,
    correct = lambda x: x.actual == x.predicted)

accuracy = pred_results.correct.sum() / pred_results.shape[0]
accuracy

0.7154150197628458

In [172]:
from sklearn.metrics import confusion_matrix, f1_score
print(confusion_matrix(test_result, tree_prediction))
f1_score(
    test_result,
    tree_prediction,
)

[[171  58]
 [ 14  10]]


0.2173913043478261